# Cortex Analyst Semantic Model Evaluator

This notebook benchmarks your semantic model by:
1. Reading test questions from a table
2. Calling Cortex Analyst to generate SQL
3. Comparing results against expected SQL
4. Storing results for tracking over time

**Run this notebook each time you update your semantic model to track accuracy improvements.**

---

## Evaluation Approach: Result Set Matching

This evaluator uses **Execution Accuracy** — we execute both the expected SQL and the generated SQL, then compare the actual result sets.

### Why This Approach?

| Approach | How It Works | Pros | Cons |
|----------|--------------|------|------|
| **Result Set Match** (this notebook) | Execute both SQLs, compare output rows | Ground truth — if results match, the answer is correct | Requires executable expected SQL |
| **SQL String Match** | Compare SQL text directly | Fast, no execution needed | Different SQL can produce identical results |
| **LLM-as-Judge** | Ask an LLM to evaluate if SQL is semantically correct | Handles ambiguity, no expected SQL needed | Subjective, can hallucinate, adds cost |

**We chose Result Set Matching because:**
- It's the most objective measure for text-to-SQL accuracy
- Cortex Analyst may generate different (but equivalent) SQL than your expected answer
- The business only cares if they get the right numbers, not how the SQL looks

### How Comparison Works

```
Expected SQL → Execute → Result Set A ─┐
                                        ├─→ Compare Values → PASS/FAIL
Generated SQL → Execute → Result Set B ─┘
```

The comparison:
- Ignores column name differences (e.g., `total_revenue` vs `TOTAL_REVENUE`)
- Ignores row ordering (unless your expected SQL has `ORDER BY`)
- Compares actual values as strings (handles numeric precision)

### When to Consider LLM-as-Judge

Use TruLens or similar LLM evaluation when:
- You want to evaluate natural language explanations (not just SQL)
- Expected SQL is hard to define or maintain
- You need metrics like "groundedness" or "relevance"

For semantic model development, Result Set Matching is recommended because it directly measures what matters: **does the user get the right answer?**

## Configuration

Update these values for your environment:

In [1]:
# === CONFIGURATION - UPDATE THESE VALUES ===

# Path to your semantic model YAML file
SEMANTIC_MODEL_PATH = "@CORTEX_AGENTS_DEMO.MAIN.SEMANTIC_MODELS/sales_orders.yaml"

# Database and schema where evaluation tables are stored
EVAL_DATABASE = "CORTEX_AGENTS_DEMO"
EVAL_SCHEMA = "CORTEX_ANALYST_EVAL"

# Optional: filter questions by category
QUESTION_CATEGORY = None  # Set to a string like "Aggregation" to filter, or None for all

## Setup

In [2]:
import json
import uuid
from datetime import datetime
import _snowflake
import streamlit as st

# Get Snowpark session
from snowflake.snowpark.context import get_active_session
session = get_active_session()

# Generate unique run ID
RUN_ID = str(uuid.uuid4())[:8]
RUN_TIMESTAMP = datetime.now()

print(f"Run ID: {RUN_ID}")
print(f"Timestamp: {RUN_TIMESTAMP}")
print(f"Semantic Model: {SEMANTIC_MODEL_PATH}")

ModuleNotFoundError: No module named '_snowflake'

## Helper Functions

In [3]:
def call_cortex_analyst(question: str) -> tuple:
    """
    Call Cortex Analyst API and return (generated_sql, error).
    """
    try:
        response = _snowflake.send_snow_api_request(
            "POST",
            "/api/v2/cortex/analyst/message",
            {},  # headers
            {},  # params
            {
                "messages": [{"role": "user", "content": [{"type": "text", "text": question}]}],
                "semantic_model_file": SEMANTIC_MODEL_PATH
            },
            {},  # request_guid
            60000  # timeout_ms
        )
        
        resp_json = json.loads(response["content"])
        
        if "message" in resp_json and "content" in resp_json["message"]:
            for item in resp_json["message"]["content"]:
                if item.get("type") == "sql":
                    return item.get("statement", ""), None
        
        return "", "No SQL in response"
    except Exception as e:
        return "", str(e)


def execute_sql_safe(sql: str) -> tuple:
    """
    Execute SQL and return (results_as_list, error).
    """
    try:
        sql = sql.strip().rstrip(';')
        df = session.sql(sql).limit(100).collect()
        results = [dict(row.asDict()) for row in df]
        return results, None
    except Exception as e:
        return None, str(e)


def compare_results(result1, result2) -> bool:
    """
    Compare two result sets (ignoring column name differences, checking values).
    """
    if result1 is None or result2 is None:
        return False
    
    if len(result1) != len(result2):
        return False
    
    def to_value_set(results):
        value_set = set()
        for row in results:
            values = tuple(sorted([str(v) for v in row.values()]))
            value_set.add(values)
        return value_set
    
    return to_value_set(result1) == to_value_set(result2)


print("✅ Helper functions loaded")

✅ Helper functions loaded


## Load Test Questions

In [4]:
# Load questions from table
query = f"""
SELECT question_id, question, expected_sql, description, category
FROM {EVAL_DATABASE}.{EVAL_SCHEMA}.EVAL_QUESTIONS
WHERE active = TRUE
"""

if QUESTION_CATEGORY:
    query += f" AND category = '{QUESTION_CATEGORY}'"

query += " ORDER BY question_id"

questions_df = session.sql(query).to_pandas()

print(f"Loaded {len(questions_df)} questions")
questions_df

NameError: name 'session' is not defined

## Run Evaluation

This cell calls Cortex Analyst for each question and compares results.

In [5]:
results = []

progress_bar = st.progress(0)
status_text = st.empty()

for idx, row in questions_df.iterrows():
    question_id = row['QUESTION_ID']
    question = row['QUESTION']
    expected_sql = row['EXPECTED_SQL']
    description = row['DESCRIPTION']
    
    status_text.text(f"[{idx+1}/{len(questions_df)}] {question[:50]}...")
    progress_bar.progress((idx + 1) / len(questions_df))
    
    # Call Cortex Analyst
    generated_sql, gen_error = call_cortex_analyst(question)
    
    if gen_error:
        results.append({
            'question_id': question_id,
            'question': question,
            'description': description,
            'expected_sql': expected_sql,
            'generated_sql': '',
            'expected_result': None,
            'generated_result': None,
            'results_match': False,
            'error': f"Analyst error: {gen_error}"
        })
        continue
    
    # Execute expected SQL
    expected_result, exp_error = execute_sql_safe(expected_sql)
    if exp_error:
        results.append({
            'question_id': question_id,
            'question': question,
            'description': description,
            'expected_sql': expected_sql,
            'generated_sql': generated_sql,
            'expected_result': None,
            'generated_result': None,
            'results_match': False,
            'error': f"Expected SQL error: {exp_error}"
        })
        continue
    
    # Execute generated SQL
    generated_result, gen_result_error = execute_sql_safe(generated_sql)
    if gen_result_error:
        results.append({
            'question_id': question_id,
            'question': question,
            'description': description,
            'expected_sql': expected_sql,
            'generated_sql': generated_sql,
            'expected_result': expected_result,
            'generated_result': None,
            'results_match': False,
            'error': f"Generated SQL error: {gen_result_error}"
        })
        continue
    
    # Compare results
    match = compare_results(expected_result, generated_result)
    
    results.append({
        'question_id': question_id,
        'question': question,
        'description': description,
        'expected_sql': expected_sql,
        'generated_sql': generated_sql,
        'expected_result': expected_result,
        'generated_result': generated_result,
        'results_match': match,
        'error': None
    })

status_text.text("✅ Evaluation complete!")
print(f"\nProcessed {len(results)} questions")

NameError: name 'st' is not defined

## Results Summary

In [6]:
passed = sum(1 for r in results if r['results_match'])
failed = sum(1 for r in results if not r['results_match'] and not r['error'])
errors = sum(1 for r in results if r['error'])
total = len(results)
accuracy = (passed / total * 100) if total > 0 else 0

# Display summary metrics
col1, col2, col3, col4 = st.columns(4)
col1.metric("Total", total)
col2.metric("Passed", passed)
col3.metric("Failed", failed)
col4.metric("Accuracy", f"{accuracy:.1f}%")

print(f"\n{'='*50}")
print(f"Run ID: {RUN_ID}")
print(f"Accuracy: {accuracy:.1f}% ({passed}/{total})")
print(f"Passed: {passed} | Failed: {failed} | Errors: {errors}")
print(f"{'='*50}")

NameError: name 'st' is not defined

## Detailed Results

In [7]:
import pandas as pd

# Create results dataframe
results_df = pd.DataFrame([{
    'Question': r['question'],
    'Status': '✅ PASS' if r['results_match'] else ('❌ ERROR' if r['error'] else '❌ FAIL'),
    'Description': r['description'],
    'Error': r['error'] or ''
} for r in results])

st.dataframe(results_df, use_container_width=True)

NameError: name 'st' is not defined

## Failed Questions Detail

Review the questions that didn't pass to identify improvements needed in your semantic model.

In [8]:
failures = [r for r in results if not r['results_match']]

if not failures:
    st.success("🎉 All questions passed!")
else:
    for i, f in enumerate(failures, 1):
        with st.expander(f"❌ {i}. {f['question']}", expanded=True):
            st.write(f"**Description:** {f['description']}")
            
            if f['error']:
                st.error(f['error'])
            
            col1, col2 = st.columns(2)
            with col1:
                st.write("**Expected SQL:**")
                st.code(f['expected_sql'], language='sql')
                if f['expected_result']:
                    st.write("**Expected Result:**")
                    st.json(f['expected_result'][:5])  # Show first 5 rows
            
            with col2:
                st.write("**Generated SQL:**")
                st.code(f['generated_sql'] or '(none)', language='sql')
                if f['generated_result']:
                    st.write("**Generated Result:**")
                    st.json(f['generated_result'][:5])  # Show first 5 rows

NameError: name 'st' is not defined

## Save Results to Table

Store this run's results for historical tracking.

In [9]:
from datetime import date, datetime

def json_serial(obj):
    if isinstance(obj, (datetime, date)):
        return obj.isoformat()
    raise TypeError(f"Type {type(obj)} not serializable")

for r in results:
    expected_json = json.dumps(r['expected_result'], default=json_serial) if r['expected_result'] else 'null'
    generated_json = json.dumps(r['generated_result'], default=json_serial) if r['generated_result'] else 'null'
    error_val = f"'{r['error']}'" if r['error'] else 'NULL'
    
    sql = f"""
    INSERT INTO {EVAL_DATABASE}.{EVAL_SCHEMA}.EVAL_RESULTS 
    (run_id, run_timestamp, semantic_model, question_id, question, 
     expected_sql, generated_sql, expected_result, generated_result, 
     results_match, execution_error)
    SELECT 
        '{RUN_ID}',
        '{RUN_TIMESTAMP}'::TIMESTAMP,
        '{SEMANTIC_MODEL_PATH}',
        {r['question_id']},
        $${r['question']}$$,
        $${r['expected_sql']}$$,
        $${r['generated_sql']}$$,
        PARSE_JSON($${expected_json}$$),
        PARSE_JSON($${generated_json}$$),
        {r['results_match']},
        {error_val}
    """
    session.sql(sql).collect()

print(f"Results saved to {EVAL_DATABASE}.{EVAL_SCHEMA}.EVAL_RESULTS")
print(f"Run ID: {RUN_ID}")

✅ Results saved to CORTEX_AGENTS_DEMO.CORTEX_ANALYST_EVAL.EVAL_RESULTS


NameError: name 'RUN_ID' is not defined

## Historical Accuracy Trend

In [10]:
# Show historical runs
history_df = session.sql(f"""
    SELECT 
        run_timestamp,
        run_id,
        accuracy_pct,
        passed,
        failed,
        errors,
        total_questions
    FROM {EVAL_DATABASE}.{EVAL_SCHEMA}.EVAL_RUN_SUMMARY
    ORDER BY run_timestamp DESC
    LIMIT 10
""").to_pandas()

if len(history_df) > 1:
    st.subheader("Accuracy Over Time")
    st.line_chart(history_df.set_index('RUN_TIMESTAMP')['ACCURACY_PCT'])

st.subheader("Recent Runs")
st.dataframe(history_df, use_container_width=True)

NameError: name 'session' is not defined

## Next Steps

Based on the failures above, consider:

1. **Add column descriptions** - If wrong columns are selected
2. **Add synonyms** - If terminology doesn't match
3. **Add sample_values** - If filter values aren't recognized
4. **Add verified queries (VQRs)** - For complex calculations
5. **Add relationships** - If joins are incorrect

After updating your semantic model, re-run this notebook to measure improvement!